<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza, Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano, Angelo Muñoz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Nicolás Becerra
- Nombre de alumno 2: Simón Sanfeliú


## Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer material del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

### **Link de repositorio de GitHub:** https://github.com/SimonSanfeliu/MDS7202-BS/tree/L9

# Importamos librerias útiles

In [2]:
!pip install -qq xgboost optuna

# 1. El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Como usted tuvo un rendimiento sobresaliente en el proyecto de caracterización de datos, Fiu lo contrata como *data scientist* de su emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv('sales.csv')
df['date'] = pd.to_datetime(df['date'])

df.head()

C:\Users\sanfe\AppData\Local\Temp\ipykernel_3380\3184305967.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'])


,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


## 1.1 Generando un Baseline (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad.
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas.
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas.
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios.
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio?
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`?
7. Guarde ambos modelos en un archivo .pkl (uno cada uno)

In [4]:
# Función para FunctionTransformer
def extract_date(df):
    """
    Genera las columnas de año, mes y día a partir de una columna de fechas.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame con la columna de fechas.

    Returns
    -------
    pd.DataFrame
        DataFrame con las columnas de año, mes y día.
    """
    # Revisando datos
    assert type(df) == pd.DataFrame
    assert "date" in df.columns

    # Generando las columnas
    df = df.assign(
        year = [d.year for d in df["date"].dt.date],
        month = [d.month for d in df["date"].dt.date],
        day = [d.day for d in df["date"].dt.date]
    )

    # Transformándolas en categorías
    df = df.astype(
        {
            "day": "category",
            "month": "category",
            "year": "category"
        }
    )

    return df

# Función para transformación logarítmica
def to_log(df_s):
    """
    Aplica una transformación logarítimica a una serie de datos.

    Parameters
    ----------
    df_s : pd.Series
        Serie de datos a transformar.

    Returns
    -------
    pd.Series
        Serie de datos transformada. 
    """
    # Revisando datos
    assert type(df_s) == pd.DataFrame

    # Transformando los datos de la serie a escala logarítmica
    df_s = df_s.apply(lambda x: np.log(x + 1))
    return df_s

In [5]:
# Obteniendo librerías necesarias
from sklearn.model_selection import train_test_split 
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, MinMaxScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error

# Definiendo la semilla
RANDOM_STATE = 42

# Separando el conjunto de datos
X_train, X_rest, y_train, y_rest = train_test_split(df, df["quantity"], test_size=.3, random_state=RANDOM_STATE)
X_val, X_test, y_val, y_test = train_test_split(X_rest, y_rest, test_size=.33, random_state=RANDOM_STATE)
X_train.drop(columns=["id", "quantity"], inplace=True)
X_val.drop(columns=["id", "quantity"], inplace=True)
X_test.drop(columns=["id", "quantity"], inplace=True)

# Separando los datos en numéricos y categóricos
num_cols = X_train.corr(numeric_only=True).columns.to_list()
cat_cols = [col for col in X_train.columns if not col in num_cols]
cat_cols.append("year")  # Agregando las nuevas columnas que se obtendrán del FunctionTransformer
cat_cols.append("month")
cat_cols.append("day")

# Atributos numéricos
num_pipe = Pipeline([
                ('Logaritmic scaler', FunctionTransformer(to_log, feature_names_out='one-to-one')),
                ('MinMax scaler', MinMaxScaler())
            ])
# Atributos categóricos
cat_pipe = Pipeline([
    ('Encoder', OneHotEncoder(sparse_output=False))
])

# Creando ColumnTransformer
ctrans = ColumnTransformer(
        transformers=[
            ("Categorico", cat_pipe, cat_cols),
            ("Numerico", num_pipe, num_cols),
        ],
        remainder="passthrough",
        verbose_feature_names_out=False
)
ctrans.set_output(transform='pandas')

ColumnTransformer(remainder='passthrough',
                  transformers=[('Categorico',
                                 Pipeline(steps=[('Encoder',
                                                  OneHotEncoder(sparse_output=False))]),
                                 ['date', 'city', 'shop', 'brand', 'container',
                                  'capacity', 'year', 'month', 'day']),
                                ('Numerico',
                                 Pipeline(steps=[('Logaritmic scaler',
                                                  FunctionTransformer(feature_names_out='one-to-one',
                                                                      func=<function to_log at 0x0000018E4D030B80>)),
                                                 ('MinMax scaler',
                                                  MinMaxScaler())]),
                                 ['lat', 'long', 'pop', 'price'])],
                  verbose_feature_names_out=False)

In [6]:
# Pipeline de entrenamiento 1
pipe_train1 = Pipeline([
    ("Date extraction", FunctionTransformer(extract_date)),
    ("Scaling", ctrans),
    ("Classifier", DummyRegressor())
])

# Entrenando
model1 = pipe_train1.fit(X_train, y_train)

# Prediciendo
y_pred1 = model1.predict(X_val)

# MAE
mae1 = mean_absolute_error(y_val, y_pred1)
print(f"MAE sobre conjunto de validación: {mae1}")

MAE sobre conjunto de validación: 13308.134750658153


> El `mean_absolute_error` es una medida de diferencia entre el valor predicho y el valor real, siendo ésta el error absoluto medio entre ambos. Lo ideal es que esta métrica sea lo más cercana a 0 posible, ya que implicaría que el valor predicho no dista mucho del real, por lo que se tendría una buena predicción. 

> Dado que el valor obtenido del MAE es alrededor de 13000, se tiene que para la pipeline generada con `DummyRegressor` es de muy baja calidad, ya que los valores predichos están muy alejados de los valores reales. Así, se tiene que el primer modelo generado es muy malo para la predicción de la demanda de cantidad de ventas, por lo que la métrica estaría diciendo que este modelo no es bueno para el negocio.

In [7]:
import xgboost as xgb

# Pipeline de entrenamiento 2
pipe_train2 = Pipeline([
    ("Date extraction", FunctionTransformer(extract_date)),
    ("Scaling", ctrans),
    ("Classifier", xgb.XGBRegressor())
])

# Entrenando
model2 = pipe_train2.fit(X_train, y_train)

# Prediciendo
y_pred2 = model2.predict(X_val)

# MAE
mae2 = mean_absolute_error(y_val, y_pred2)
print(f"MAE sobre conjunto de validación: {mae2}")

MAE sobre conjunto de validación: 2500.3221446955317


> Al cambiar el `DummyRegressor` por el `XGBRegressor`, se tiene una mejora instantánea en la métrica, teniendo ahora un MAE de cerca de 2500, lo que es mucho más cercano a 0 que 13000. Este valor sigue indicando que el clasificador es malo para el negocio, pero muestra una inmediata mejora con el anterior.

In [8]:
import os
import joblib

# Creando la carpeta de los modelos si es que no existe
if not os.path.exists("models"):
    os.mkdir("models")

# Guardando los archivos en pkls separados
joblib.dump(model1, "models/dummy_model.pkl")
joblib.dump(model2, "models/default_xgb_model.pkl")

['models/default_xgb_model.pkl']

## 1.2 Forzando relaciones entre parámetros con XGBoost (1.0 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo.

Vuelva a entrenar el `Pipeline`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo?

Nuevamente, guarde su modelo en un archivo .pkl

Nota: Para realizar esta parte, debe apoyarse en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>.

Hint: Para implementar el constraint, se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

In [9]:
# Pipeline de entrenamiento 3
pipe_train3 = Pipeline([
    ("Date extraction", FunctionTransformer(extract_date)),
    ("Scaling", ctrans),
    ("Classifier", xgb.XGBRegressor(monotone_constraints={'price': -1}))
])

# Entrenando
model3 = pipe_train3.fit(X_train, y_train)

# Prediciendo
y_pred3 = model3.predict(X_val)

# MAE
mae3 = mean_absolute_error(y_val, y_pred3)
print(f"MAE sobre conjunto de validación: {mae3}")

MAE sobre conjunto de validación: 2707.9117589321568


> Al incluir esta relación, el error aumentó, lo que indica que empeoró el modelo. Esto es esperable, puesto que, mientras que es cierto que la demanda por lo general disminuye al momento de aumentar los precios, esta relación no es lineal. Esto quiere decir que no es directa la relación inversa que tienen, por lo que al definirla así, se tiene que el modelo empeorará ya que no es el comportamiento real de la demanda el que se estaría modelando, sino que una simplificación muy grande de éste.

In [10]:
# Guardando el modelo
joblib.dump(model3, "models/constraint_xgb_model.pkl")

['models/constraint_xgb_model.pkl']

## 1.3 Optimización de Hiperparámetros con Optuna (2.0 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se le pide:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)
- Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados?
- Fije el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

**Explicaciones de los hiperparámetros**

* `min_frequency`: Es la frecuencia mínima que debe tener una categoría para que se considere en el modelo. Al ser un flotante, corresponde al porcentaje de frecuencia mínimo para considerarlo y se obtiene multiplicando este parámetro con el número de muestras.
* `learning_rate`: El learning rate corresponde al tamaño de paso que se da en cada iteración del algoritmo. Este valor es multiplicado por el gradiente de la función de pérdida, por lo que se tiene que, mientras más pequeño sea este valor, más pequeño será el paso que se da en cada iteración y más tardará el entrenamiento. El rango es adecuado para este hiperparámetro, siendo 0.1 un máximo bien razonable dentro del entrenamiento.
* `n_estimators`: El número de estimadores corresponde a la cantidad de árboles que se construyen en el algoritmo. De esta manera, se tiene que, mientras más grande sea este valor, más complejo será el modelo y más tardará el entrenamiento. El rango hace sentido ya que son enteros, 
* `max_depth`: Es la profundidad de cada árbol. En cierta forma, es la cantidad de preguntas que se le hacen al modelo para que éste pueda predecir. La profundidad entre 3 y 10 resulta coherente, ya que se tiene que, mientras más profundo sea el árbol, más complejo será el modelo y más tardará el entrenamiento, por lo que 10 resulta razonable como un máximo de capas.
* `max_leaves`: Corresponde al número máximo de hojas del árbol resultante. De cierta manera, regula la complejidad del modelo al igual que max_depth, pero de una manera más directa. El rango es razonable; sin embargo, se recomienda que sea menor a 64 para no ralentizar el entrenamiento demasiado.
* `min_child_weight`: Es el peso mínimo que debe tener un nodo para que se divida. Este hiperparámetro regula la complejidad del modelo, ya que mientras más grande sea este valor, más nodos se descartarán y más simple será el modelo. Debido a la naturaleza del modelo, el rango es adecuado, aunque incluso existen algunos modelos donde 200 sea un valor razonable.
* `reg_alpha`: Regularización L1 a los pesos de los nodos. Este penaliza el valor absoluto de los pesos o coeficientes, ayudando a hacer el modelo más robusto y evitar el esparcimiento al promover que algunos pesos de anulen al ser menos importantes. El rango de este varia entre 0 y 1, siendo 0 una penalización nula.
* `reg_lambda`: Regularización L2 a los pesos de los nodos. Este penaliza el valor cuadrático de los pesos o coeficientes, lo que ayuda a prevenir el overfitting al suavizar los pesos que se escapan del resto, limitando la magnitud de ellos. El rango de este varia entre 0 y 1, siendo 0 una penalización nula, al igual que el anterior.

In [11]:
# Creando la optimización con Optuna
import optuna

def objective(trial):
    """
    Función objetivo para optimizar los hiperparámetros del modelo. Utiliza la libreria Optuna.

    Parameters
    ----------
    trial : optuna.trial._trial.Trial
        Objeto de la clase Trial de Optuna.

    Returns
    -------
    mae_opt : float
        Error absoluto medio del modelo optimizado. Es el valor que se intenta minimizar.
    """
    # Agregando valores a probar
    min_frequency = trial.suggest_float('min_frequency', 0, 1)
    learning_rate = trial.suggest_float('learning_rate', 1e-3, 1e-1)
    n_estimators = trial.suggest_int('n_estimators', 50, 1000)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    max_leaves = trial.suggest_int('max_leaves', 0, 100)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 5)
    reg_alpha = trial.suggest_float('reg_alpha', 0, 1)
    reg_lambda = trial.suggest_float('reg_lambda', 0, 1)

    # Re-editando el OneHotEncoder de ColumnTransformer
    cat_pipe = Pipeline([
        ('Encoder', OneHotEncoder(min_frequency=min_frequency, sparse_output=False))
    ])

    ctrans = ColumnTransformer(
            transformers=[
                ("Categorico", cat_pipe, cat_cols),
                ("Numerico", num_pipe, num_cols),
            ],
            remainder="passthrough",
            verbose_feature_names_out=False
    )
    ctrans.set_output(transform='pandas')

    # Mejor de los 3 pipelines anteriores: pipeline 2
    pipe_train2 = Pipeline([
        ("Date extraction", FunctionTransformer(extract_date)),
        ("Scaling", ctrans),
        ("Classifier", xgb.XGBRegressor(learning_rate=learning_rate,
                                        n_estimators=n_estimators,
                                        max_depth=max_depth,
                                        max_leaves=max_leaves,
                                        min_child_weight=min_child_weight,
                                        reg_alpha=reg_alpha,
                                        reg_lambda=reg_lambda))
    ])

    # Entrenando el pipeline
    model_opt = pipe_train2.fit(X_train, y_train)

    # Prediciendo
    y_pred_opt = model_opt.predict(X_val)

    # Obteniendo el MAE
    mae_opt = mean_absolute_error(y_val, y_pred_opt)

    return mae_opt

C:\Users\sanfe\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
# Seteando semilla de optuna
optuna_seed = 0

# Sampler
sampler = optuna.samplers.TPESampler(seed=optuna_seed)

# Generando el estudio de hiperparámetros
study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(objective, timeout=300)  # 5 min de entrenamiento

[I 2023-11-16 23:24:32,435] A new study created in memory with name: no-name-925c7de2-9857-47d5-961d-cb07ebe0b09a
[I 2023-11-16 23:24:36,697] Trial 0 finished with value: 9126.717615023155 and parameters: {'min_frequency': 0.5488135039273248, 'learning_rate': 0.07180374727086954, 'n_estimators': 623, 'max_depth': 7, 'max_leaves': 42, 'min_child_weight': 4, 'reg_alpha': 0.4375872112626925, 'reg_lambda': 0.8917730007820798}. Best is trial 0 with value: 9126.717615023155.
[I 2023-11-16 23:24:40,846] Trial 1 finished with value: 8930.980045333883 and parameters: {'min_frequency': 0.9636627605010293, 'learning_rate': 0.038960710363751996, 'n_estimators': 802, 'max_depth': 7, 'max_leaves': 57, 'min_child_weight': 5, 'reg_alpha': 0.07103605819788694, 'reg_lambda': 0.08712929970154071}. Best is trial 1 with value: 8930.980045333883.
[I 2023-11-16 23:24:48,475] Trial 2 finished with value: 2017.3263964582986 and parameters: {'min_frequency': 0.02021839744032572, 'learning_rate': 0.0834293647092

In [22]:
# Obteniendo la cantidad de trials, el mejor MAE y los mejores hiperparámetros
n_trails = len(study.trials)
best_mae = study.best_value
best_hp = study.best_params
print(f"Cantidad de trials hechos: {n_trails} \nMejor MAE obtenido: {best_mae} \nMejores hiperparámetros: {best_hp}")

Cantidad de trials hechos: 51 
Mejor MAE obtenido: 1951.8277633524388 
Mejores hiperparámetros: {'min_frequency': 0.07635478834392678, 'learning_rate': 0.09032442930563103, 'n_estimators': 913, 'max_depth': 8, 'max_leaves': 83, 'min_child_weight': 3, 'reg_alpha': 0.5590535826737515, 'reg_lambda': 0.6940799730661463}


> El MAE mejoró considerablemente, lo que indica que el modelo es mucho mejor que el anterior. Esto se debe a que se optimizaron los hiperparámetros, complejizando el modelo y la interrelación de los parámetros con el entrenamiento, adecuandolo a los datos. Así, se tiene que el modelo es mucho mejor que el anterior, ya que se optimizaron los hiperparámetros para que se ajusten mejor a los datos.

In [14]:
# Obteniendo el mejor modelo
cat_pipe_best = Pipeline([
    ('Encoder', OneHotEncoder(min_frequency=best_hp['min_frequency'], sparse_output=False))
])

ctrans_best = ColumnTransformer(
        transformers=[
            ("Categorico", cat_pipe_best, cat_cols),
            ("Numerico", num_pipe, num_cols),
        ],
        remainder="passthrough",
        verbose_feature_names_out=False
)
ctrans_best.set_output(transform='pandas')

# Mejor pipeline
pipe_train_best = Pipeline([
    ("Date extraction", FunctionTransformer(extract_date)),
    ("Scaling", ctrans_best),
    ("Classifier", xgb.XGBRegressor(learning_rate=best_hp['learning_rate'],
                                    n_estimators=best_hp['n_estimators'],
                                    max_depth=best_hp['max_depth'],
                                    max_leaves=best_hp['max_leaves'],
                                    min_child_weight=best_hp['min_child_weight'],
                                    reg_alpha=best_hp['reg_alpha'],
                                    reg_lambda=best_hp['reg_lambda']))
])

# Entrenando el mejor pipeline
model_best = pipe_train_best.fit(X_train, y_train)

# Guardando el mejor modelo
joblib.dump(model_best, "models/best_xgb_model1.pkl")

['models/best_xgb_model1.pkl']

## 1.4 Optimización de Hiperparámetros con Optuna y Prunners (1.7)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento?
- Utilizar `optuna.integration.XGBoostPruningCallback` como método de **Prunning**
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

> Prunning o "poda" es una forma de simplificar o reducir la estructura de un modelo con el fin de mejorar su rendimiento. Esto se hace eliminando nodos o ramas del modelo, lo que ayuda a reducir el overfitting y la complejidad del modelo. Al hacer esto, se espera que la capacidad de generalización frente a datos nuevos sea aun mayor y, por otro lado, mejorar la eficiencia computacional del modelo al hacerlo "menos costoso" de entrenar.

In [15]:
# Creando la optimización con Optuna

def objective2(trial):
    """
    Función objetivo para optimizar los hiperparámetros del modelo. Utiliza la libreria Optuna y método Prunning.

    Parameters
    ----------
    trial : optuna.trial._trial.Trial
        Objeto de la clase Trial de Optuna.

    Returns
    -------
    mae_opt : float
        Error absoluto medio del modelo optimizado. Es el valor que se intenta minimizar.
    """
    # Agregando valores a probar
    min_frequency = trial.suggest_float('min_frequency', 0, 1)
    learning_rate = trial.suggest_float('learning_rate', 1e-3, 1e-1)
    n_estimators = trial.suggest_int('n_estimators', 50, 1000)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    max_leaves = trial.suggest_int('max_leaves', 0, 100)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 5)
    reg_alpha = trial.suggest_float('reg_alpha', 0, 1)
    reg_lambda = trial.suggest_float('reg_lambda', 0, 1)

    # Re-editando el OneHotEncoder de ColumnTransformer
    cat_pipe = Pipeline([
        ('Encoder', OneHotEncoder(min_frequency=min_frequency, sparse_output=False))
    ])

    ctrans = ColumnTransformer(
            transformers=[
                ("Categorico", cat_pipe, cat_cols),
                ("Numerico", num_pipe, num_cols),
            ],
            remainder="passthrough",
            verbose_feature_names_out=False
    )
    ctrans.set_output(transform='pandas')

    # Definiendo el prunning
    pruning_callback = optuna.integration.XGBoostPruningCallback(
        trial, observation_key="validation_1-mae"
    )

    # Mejor de los 3 pipelines anteriores: pipeline 2
    pipe_train2 = Pipeline([
        ("Date extraction", FunctionTransformer(extract_date)),
        ("Scaling", ctrans),
        ("Classifier", xgb.XGBRegressor(learning_rate=learning_rate,
                                        n_estimators=n_estimators,
                                        max_depth=max_depth,
                                        max_leaves=max_leaves,
                                        min_child_weight=min_child_weight,
                                        reg_alpha=reg_alpha,
                                        reg_lambda=reg_lambda,
                                        eval_metric='mae',
                                        early_stopping_rounds=10,
                                        callbacks=[pruning_callback]))
    ])

    # Paso necesario
    pipe_dumb = Pipeline([
        ("Date extraction", FunctionTransformer(extract_date)),
        ("Scaling", ctrans)
    ])
    step = pipe_dumb.fit(X_train, y_train)
    X_train2 = step.transform(X_train)
    X_val2 = step.transform(X_val)

    # Entrenando el pipeline
    model_opt = pipe_train2.fit(X_train,
                                y_train,
                                Classifier__eval_set=[(X_train2, y_train), (X_val2, y_val)],
                                Classifier__verbose=False)

    # Prediciendo
    y_pred_opt = model_opt.predict(X_val)

    # Obteniendo el MAE
    mae_opt = mean_absolute_error(y_val, y_pred_opt)

    return mae_opt

In [16]:
# Quitando los prints de prunning
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Generando el estudio de hiperparámetros
study2 = optuna.create_study(direction='minimize', sampler=sampler)
study2.optimize(objective2, timeout=300, show_progress_bar=True)  # 5 min de entrenamiento

   0%|          | 00:00/05:00

Best trial: 12. Best value: 2029.99:  100%|██████████| 05:00/05:00


In [17]:
# Obteniendo la cantidad de trials, el mejor MAE y los mejores hiperparámetros
n_trails2 = len(study2.trials)
best_mae2 = study2.best_value
best_hp2 = study2.best_params
print(f"Cantidad de trials hechos: {n_trails2} \nMejor MAE obtenido: {best_mae2} \nMejores hiperparámetros: {best_hp2}")

Cantidad de trials hechos: 129 
Mejor MAE obtenido: 2029.9910766438584 
Mejores hiperparámetros: {'min_frequency': 0.03402343044784076, 'learning_rate': 0.09943651925920696, 'n_estimators': 415, 'max_depth': 10, 'max_leaves': 100, 'min_child_weight': 1, 'reg_alpha': 0.2104797919643507, 'reg_lambda': 0.03098980392770021}


> El MAE empeoró levemente en comparación al modelo optimizado anteriormente, ya que aumentó. Mientras que se espera que al quitar pruebas se evite el overfitting y así tener una mejora del modelo, la variabilidad de los hiperparámetros de forma aleatoria conlleva a que se tenga también mayor variabilidad al momento de obtener su MAE. Al ir quitando pruebas, se van generando cada vez más trials, lo que aumenta la variabilidad de los parámetors, abriendo más posibilidades de resultados, lo que puede mejorar o empeorar el modelo. Para esta corrida, se empeoró.

In [18]:
# Obteniendo el mejor modelo
cat_pipe_best2 = Pipeline([
    ('Encoder', OneHotEncoder(min_frequency=best_hp2['min_frequency'], sparse_output=False))
])

ctrans_best2 = ColumnTransformer(
        transformers=[
            ("Categorico", cat_pipe_best2, cat_cols),
            ("Numerico", num_pipe, num_cols),
        ],
        remainder="passthrough",
        verbose_feature_names_out=False
)
ctrans_best2.set_output(transform='pandas')

# Mejor pipeline
pipe_train_best2 = Pipeline([
    ("Date extraction", FunctionTransformer(extract_date)),
    ("Scaling", ctrans_best2),
    ("Classifier", xgb.XGBRegressor(learning_rate=best_hp2['learning_rate'],
                                    n_estimators=best_hp2['n_estimators'],
                                    max_depth=best_hp2['max_depth'],
                                    max_leaves=best_hp2['max_leaves'],
                                    min_child_weight=best_hp2['min_child_weight'],
                                    reg_alpha=best_hp2['reg_alpha'],
                                    reg_lambda=best_hp2['reg_lambda']))
])

# Entrenando el mejor pipeline
model_best2 = pipe_train_best2.fit(X_train, y_train)

# Guardando el mejor modelo
joblib.dump(model_best2, "models/best_xgb_model2.pkl")

['models/best_xgb_model2.pkl']

## 1.5 Visualizaciones (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

- Gráfico de historial de optimización
- Gráfico de coordenadas paralelas
- Gráfico de importancia de hiperparámetros

Comente sus resultados: ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo?

In [19]:
# Obteniendo las herramientas gráficas
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_param_importances

# Gráfico de historial de optimización
plot_optimization_history(study2, target_name="MAE")

> Viendo el gráfico, es posible ver mejoras notables desde el trial número 10. Mientras que hay una tendencia a subir el MAE a alrededor de los 6000, éste oscila también en los 2000, donde se encuentra el mejor valor.

In [20]:
# Gráfico de coordenadas paralelas
plot_parallel_coordinate(study2, target_name="MAE")

> De derecha a izquierda, se puede observar que los primeros hiperparámetros tienen un comportamiento más errático (se prueban valores más distintos entre sí), pero a medida que se avanza, se ve que algunos parámetros convergen a ciertos valores. Por ejemplo, `min_frequency` converge a valores menores que 0.1, `max_leaves` a valores sobre 80 y `learning_rate` a valores sobre 0.9. Este comportamiento se ve cada vez más pronunciado (es más fuerte la convergencia a estos valores) mientras más a la izquierda se esté llegando, coincidiendo con la convegencia d elos valores de la métrica de evaluación MAE. 

In [21]:
# Gráfico de importancia de hiperparámetros
plot_param_importances(study2, target_name="MAE")

> El parámetro más importante por lejos es el `min_frequency`, puesto que define cómo serán los datos del pipeline a través del `OneHotEncoder`. Esto conlleva a que defina el resto de los parámetros del pipeline, ya que éste se ajustará a los datos, los cuales dependerán de `min_frequency`.

## 1.6 Síntesis de resultados (0.3)

Finalmente, genere una tabla resumen del MAE obtenido en los 5 modelos entrenados (desde Baseline hasta XGBoost con Constraints, Optuna y Prunning) y compare sus resultados. ¿Qué modelo obtiene el mejor rendimiento? 

Por último, cargue el mejor modelo, prediga sobre el conjunto de test y reporte su MAE. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto?

In [23]:
from IPython.display import display

# Generando tabla resumen
summary = {"Modelos": ["Baseline", "XGBoost", "XGBoost Constraints", "XGBoost Optuna", "XGBoost Prunning"],
           "MAE": [mae1, mae2, mae3, best_mae, best_mae2]}

df_sum = pd.DataFrame(summary)
display(df_sum)

,Modelos,MAE
0,Baseline,13308.134751
1,XGBoost,2500.322145
2,XGBoost Constraints,2707.911759
3,XGBoost Optuna,1951.827763
4,XGBoost Prunning,2029.991077


> Comentario XD

In [26]:
# Mejor modelo: XGBoost Optuna
best_model = joblib.load("models/best_xgb_model1.pkl")

# Prediciendo en test
best_y_pred = best_model.predict(X_test)

# Obteniendo su MAE
best_mae_final = mean_absolute_error(y_test, best_y_pred)
print(f"MAE del mejor modelo en el conjunto de prueba: {best_mae_final}")

MAE del mejor modelo en el conjunto de prueba: 2000.6314546949002


# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>